In [1]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
import pickle

In [2]:
# model = Word2Vec.load("../models/b25-sn-v256/b25-sn-v256.model")
# model = Word2Vec.load("../models/b25-sn-v512/b25-sn-v512-b.model")  # <--
# model = Word2Vec.load("../models/b25-sn-v512/b25-sn-v512-a.model")
# model = Word2Vec.load("../models/b25-sn-v50/b25-sn-v50.model")
# model = Word2Vec.load("../models/b25-sn-v256/b25-sn-v256-d-c.model")
model = Word2Vec.load("../models/b25-sn-v512/b25-sn-v512-b-d.model")
# model = Word2Vec.load("../models/b25-sn-v256/b25-sn-v256-d-c.model")

In [2]:
# cboe
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath("../"))  # Adjust based on the notebook's location

# Now import the `cboe` package
from cboe import *

model = Entity2Vec.load("../models/b25-sn-v256/b25-sn-v256-e.pkl")

In [2]:
from algo_testing import son2vecgl_a

with open(
    "../models/b25-sn-v512/b25-sn-v512-g.pkl", "rb"
) as file:  # "rb" means read in binary mode
    model = pickle.load(file)

In [3]:
with open("../data/tokenized_data/playlist_names/dataset_test_v3.pkl", "rb") as f:
    tokenized_playlists = pickle.load(f)

## Precision@1

In [5]:
tested = 0
correct = 0

test_set = tokenized_playlists[:1000]
a = 0
for playlist in test_set:
    a += len(playlist)
print(f"Total songs: {a}")

vgl_a = 0
vgl_b = 0

for playlist in test_set:
    for song in playlist:
        vgl_a += 1

        try:

            similar_words = model.wv.most_similar(song, topn=1)
            # similar_words = model.nearest(song, k=1)
            # similar_words = model.nearest(song, k=1)
            if similar_words == []:
                continue

            tested += 1

            if any(word[0] in playlist for word in similar_words):
                correct += 1
        except:
            vgl_b += 1
            continue


print(f"correct: {correct}")
print(f"Accuracy: {correct/tested}")
print(f"Tested: {vgl_a}")
print(f"Wrong: {vgl_b}")

# this is a good value to see here for the data
# over 200
# 0.6791003839824465
#

# for the 256 model with 40 epochs
# Total songs: 29290
# correct: 20505
# Accuracy: 0.700068282690338
# Tested: 29290
# Wrong: 0

# for the 512 model  with 40 epochs
# Total songs: 29290
# correct: 20422
# Accuracy: 0.697234551041311
# Tested: 29290
# Wrong: 0

Total songs: 29290
correct: 20422
Accuracy: 0.697234551041311
Tested: 29290
Wrong: 0


## Precision@K

In [7]:
tested = 0
correct = 0
K = 3

test_set = tokenized_playlists[:250]
a = 0
for playlist in test_set:
    a += len(playlist)
print(f"Total songs: {a}")

vgl_a = 0
vgl_b = 0

for playlist in test_set:
    for song in playlist:
        vgl_a += 1

        try:

            similar_words = model.wv.most_similar(song, topn=K)
            # similar_words = model.nearest(song, k=K)
            if similar_words == []:
                continue

            tested += 1

            for similar_word in similar_words:
                if similar_word[0] in playlist:
                    correct += 1
        except:
            vgl_b += 1
            continue


print(f"correct: {correct}")
print(f"Accuracy: {correct/(tested*K)}")
print(f"Tested: {vgl_a}")
print(f"Wrong: {vgl_b}")

Total songs: 9713
correct: 9375
Accuracy: 0.32173375887985173
Tested: 9713
Wrong: 0


## Recall@K

In [ ]:
# recall @ k would be the same as precision at k with a threshold for the k results...

# Second Test with the Combined Artist and Trackname 
to exclude songs named the same with different vibe

#### b25-sn-v50 param: vector_size=50, window=5, min_count=1, sg=0
Accuracy: 0.36720959719106605

#### b25-sn-v256-a param: vector_size=256, window=5, min_count=1, sg=0
Accuracy: 0.36691699990246757

#### b25-sn-v256-b param: vector_size=256, window=10, min_count=1, sg=0
Accuracy: 0.43333658441431777

#### b25-sn-v256-c param: vector_size=256, window=20, min_count=1, sg=0
Accuracy: 0.4426996976494684

#### b25-sn-v256-d param: vector_size=256, window=20, min_count=1, sg=1, ns_exponent=0.0
Accuracy: 0.6513215644201698

#### b25-sn-v512-a param: vector_size=512, window=100, min_count=1, sg=0
Accuracy: 0.5702721154783965

#### b25-sn-v512-b param: vector_size=512, window=100, min_count=1, sg=1, ns_exponent=0.0
Accuracy: 0.773919828342924

#### b25-sn-v512-d - CBOS param: vector_size=512, epochs=5, learning_rate=0.015*

Accuracy: 0.08009367681498829

#### b25-sn-v512-e param: vector_size: 512, window=inf, min_count=1, epochs=15, learning_rate=0.025, algo=CBOS-d
Accuracy: 0.052830188679245285
* Used distance to add a score to the gradient

#### b25-snv128-f param: vector_size: 128, window=inf, min_count=1, epochs=15, learning_rate=0.025, algo=GLSE
Accuracy: 0.0038022813688212928



\* trained with only 3.5% of the training data 

## Unit tested Results

#### b25-sn-v50 param: vector_size=50, window=5, min_count=1, sg=0
Accuracy: 0.260704184141227

#### b25-sn-v256-a param: vector_size=256, window=5, min_count=1, sg=0
Accuracy: 0.28089339705452065

#### b25-sn-v256-b param: vector_size=256, window=10, min_count=1, sg=0
Accuracy: 0.3198088364381157

#### b25-sn-v256-c param: vector_size=256, window=20, min_count=1, sg=0
Accuracy: 0.39529893689651807

#### b25-sn-v256-d param: vector_size=256, window=20, min_count=1, sg=1, ns_exponent=0.0
Accuracy: 0.4845411099190481

#### b25-sn-v512-a param: vector_size=512, window=100, min_count=1, sg=0
Accuracy: 0.5000487662147665

#### b25-sn-v512-b param: vector_size=512, window=100, min_count=1, sg=1, ns_exponent=0.0
Accuracy: 0.6720959719106603


